## Step 5.1: Environment Setup and Configuration

In [ ]:
# Environment setup
import os
import sys
import time
import json
import asyncio
from datetime import datetime
from typing import Dict, List, Optional, Any
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Add parent directory for module imports
sys.path.append(os.path.dirname(os.getcwd()))

# Load environment variables
load_dotenv()

print("✅ Environment setup complete")
print(f"Working directory: {os.getcwd()}")
print(f"Python version: {sys.version}")

## Step 5.2: Import Agent Framework Hybrid Router

Import the modern Agent Framework-based hybrid router with built-in context management.

In [ ]:
# Import the Agent Framework hybrid router
import importlib
import sys

# Force reload of the module if it's already loaded
if 'modules.hybrid_router_agent_framework' in sys.modules:
    importlib.reload(sys.modules['modules.hybrid_router_agent_framework'])

try:
    from modules.hybrid_router_agent_framework import (
        HybridAgentRouter, 
        HybridAgentRouterConfig,
        create_hybrid_agent_router_from_env
    )
    router_available = True
    print("✅ Agent Framework hybrid router imported successfully")
except ImportError as e:
    router_available = False
    print(f"❌ Agent Framework hybrid router not available: {e}")
    print("Please ensure modules are properly configured")

# Initialize router from environment
if router_available:
    try:
        # Create router with unique session
        session_id = f"lab5_session_{int(time.time())}"
        hybrid_router = create_hybrid_agent_router_from_env(session_id=session_id)
        
        print(f"✅ Agent Framework hybrid router initialized")
        print(f"   Session ID: {hybrid_router.context_manager.session_id}")
        
        # Show system capabilities
        capabilities = hybrid_router.get_system_capabilities()
        print(f"\n📊 Available routing targets:")
        for target, available in capabilities['available_targets'].items():
            status = "✅" if available else "❌"
            print(f"   {target}: {status}")
        
        print(f"\n🤖 Agent Framework:")
        af_info = capabilities.get('agent_framework', {})
        print(f"   Available: {af_info.get('available', False)}")
        print(f"   Endpoint: {af_info.get('endpoint', 'N/A')}")
        print(f"   Model: {af_info.get('model', 'N/A')}")
        
        print(f"\n🧠 ML Routers:")
        ml_info = capabilities.get('ml_routers', {})
        print(f"   BERT: {ml_info.get('bert_router', False)}")
        print(f"   PHI: {ml_info.get('phi_router', False)}")
        
    except Exception as e:
        print(f"⚠️ Failed to initialize router: {e}")
        import traceback
        traceback.print_exc()
        hybrid_router = None
        router_available = False
else:
    hybrid_router = None

## Step 5.3: Enhanced Conversation Manager with Agent Framework

Create an advanced conversation manager that leverages the HybridAgentRouter's built-in ConversationContextManager.

In [ ]:
class AgentFrameworkConversationManager:
    """
    Advanced conversation manager for Agent Framework hybrid routing.
    Leverages built-in ConversationContextManager for context handling.
    """
    
    def __init__(self, hybrid_router: HybridAgentRouter, max_exchanges: int = 20):
        """
        Initialize conversation manager.
        
        Args:
            hybrid_router: HybridAgentRouter instance with Agent Framework
            max_exchanges: Maximum conversation exchanges to track
        """
        self.router = hybrid_router
        self.max_exchanges = max_exchanges
        
        # Access the router's built-in context manager
        self.context_manager = hybrid_router.context_manager
        self.session_id = self.context_manager.session_id
        
        # Conversation analytics
        self.conversation_stats = {
            'total_exchanges': 0,
            'local_responses': 0,
            'agent_responses': 0,
            'apim_responses': 0,
            'azure_responses': 0,
            'model_switches': 0,
            'total_time': 0.0,
            'session_start': datetime.now(),
            'last_source': None
        }
        
        print(f"🎭 Agent Framework Conversation Manager initialized")
        print(f"   Session ID: {self.session_id}")
        print(f"   Max exchanges: {max_exchanges}")
        print(f"   Context manager: ✅ Built-in from HybridAgentRouter")
    
    async def chat(
        self, 
        user_message: str, 
        use_context: bool = True,
        show_reasoning: bool = False
    ) -> Dict[str, Any]:
        """
        Process a chat message through the hybrid routing system.
        
        Args:
            user_message: User's message
            use_context: Whether to include conversation context
            show_reasoning: Show routing analysis
            
        Returns:
            Dictionary with response and metadata
        """
        start_time = time.time()
        
        # Route query through Agent Framework hybrid router (async)
        result = await self.router.route_async(
            query=user_message,
            use_context=use_context,
            show_reasoning=show_reasoning
        )
        
        response_time = time.time() - start_time
        
        # Update statistics
        self.conversation_stats['total_exchanges'] += 1
        self.conversation_stats['total_time'] += response_time
        
        source = result.get('source', 'unknown')
        
        # Track routing target
        if 'local' in source:
            self.conversation_stats['local_responses'] += 1
        elif 'agent' in source:
            self.conversation_stats['agent_responses'] += 1
        elif 'apim' in source:
            self.conversation_stats['apim_responses'] += 1
        elif 'azure' in source:
            self.conversation_stats['azure_responses'] += 1
        
        # Track model switches
        if self.conversation_stats['last_source'] and self.conversation_stats['last_source'] != source:
            self.conversation_stats['model_switches'] += 1
        
        self.conversation_stats['last_source'] = source
        
        # Format result
        return {
            'user_message': user_message,
            'assistant_response': result.get('response', ''),
            'source': source,
            'response_time': response_time,
            'success': result.get('success', False),
            'context_used': use_context,
            'analysis': result.get('analysis', {}),
            'exchange_number': self.conversation_stats['total_exchanges']
        }
    
    def get_conversation_history(self, count: int = 10) -> List[Dict]:
        """Get recent conversation history from context manager."""
        return self.context_manager.get_recent_exchanges(count)
    
    def get_session_summary(self) -> Dict[str, Any]:
        """Get detailed session summary with analytics."""
        total = self.conversation_stats['total_exchanges']
        
        if total == 0:
            return {'message': 'No conversations yet'}
        
        avg_time = self.conversation_stats['total_time'] / total
        
        # Calculate routing distribution
        local_pct = (self.conversation_stats['local_responses'] / total) * 100
        agent_pct = (self.conversation_stats['agent_responses'] / total) * 100
        apim_pct = (self.conversation_stats['apim_responses'] / total) * 100
        azure_pct = (self.conversation_stats['azure_responses'] / total) * 100
        
        return {
            'session_id': self.session_id,
            'total_exchanges': total,
            'routing_distribution': {
                'local': {
                    'count': self.conversation_stats['local_responses'],
                    'percentage': local_pct
                },
                'agent': {
                    'count': self.conversation_stats['agent_responses'],
                    'percentage': agent_pct
                },
                'apim': {
                    'count': self.conversation_stats['apim_responses'],
                    'percentage': apim_pct
                },
                'azure': {
                    'count': self.conversation_stats['azure_responses'],
                    'percentage': azure_pct
                }
            },
            'model_switches': self.conversation_stats['model_switches'],
            'average_response_time': avg_time,
            'total_time': self.conversation_stats['total_time'],
            'session_duration': (datetime.now() - self.conversation_stats['session_start']).total_seconds()
        }
    
    def clear_context(self):
        """Clear conversation context."""
        self.context_manager.clear_history()
        print(f"🧹 Conversation context cleared for session {self.session_id}")
    
    def export_conversation(self, filename: str = None) -> str:
        """Export conversation history to JSON."""
        return self.context_manager.export_conversation(filename)
    
    def display_conversation_analytics(self):
        """Display formatted conversation analytics."""
        summary = self.get_session_summary()
        
        if 'message' in summary:
            print(summary['message'])
            return
        
        print("\n" + "="*70)
        print("📊 CONVERSATION ANALYTICS")
        print("="*70)
        print(f"Session ID: {summary['session_id']}")
        print(f"Total Exchanges: {summary['total_exchanges']}")
        print(f"Session Duration: {summary['session_duration']:.1f}s")
        print(f"Average Response Time: {summary['average_response_time']:.3f}s")
        print(f"Model Switches: {summary['model_switches']}")
        
        print(f"\n📈 Routing Distribution:")
        for target, data in summary['routing_distribution'].items():
            if data['count'] > 0:
                print(f"   {target.upper()}: {data['count']} ({data['percentage']:.1f}%)")
        
        print("="*70)

# Initialize conversation manager if router is available
if router_available and hybrid_router:
    conversation_manager = AgentFrameworkConversationManager(hybrid_router)
    print("\n✅ Conversation manager ready for multi-turn conversations!")
else:
    conversation_manager = None
    print("\n⚠️ Conversation manager not available (router initialization failed)")

## Step 5.4: Test Multi-Turn Conversation Scenarios

Test various conversation scenarios to demonstrate intelligent routing and context preservation.

In [ ]:
async def test_conversation_scenarios():
    """Test various multi-turn conversation scenarios."""
    
    if not conversation_manager:
        print("❌ Conversation manager not available")
        return
    
    print("🧪 Testing Multi-Turn Conversation Scenarios with Agent Framework")
    print("="*70)
    
    # Scenario 1: Basic connectivity test
    print("\n" + "="*70)
    print("📝 SCENARIO 1: Basic Connectivity Test")
    print("="*70)
    
    scenario1_exchanges = [
        "Hello, can you help me?",
        "What can you do?",
        "That's great, thanks!"
    ]
    
    for i, message in enumerate(scenario1_exchanges, 1):
        print(f"\n💬 User ({i}/{len(scenario1_exchanges)}): {message}")
        
        result = await conversation_manager.chat(
            user_message=message,
            use_context=True,
            show_reasoning=False
        )
        
        print(f"🤖 Assistant [{result['source']}] ({result['response_time']:.3f}s):")
        print(f"   {result['assistant_response'][:200]}...")
        
        if not result['success']:
            print(f"   ⚠️ Response failed")
    
    # Scenario 2: Simple to complex progression
    print("\n" + "="*70)
    print("📝 SCENARIO 2: Simple to Complex Progression")
    print("="*70)
    
    scenario2_exchanges = [
        "Hi there!",
        "What is Python?",
        "Can you analyze the pros and cons of using Python for enterprise application development?",
        "Based on that analysis, what would you recommend for a large-scale financial services platform?"
    ]
    
    for i, message in enumerate(scenario2_exchanges, 1):
        print(f"\n💬 User ({i}/{len(scenario2_exchanges)}): {message}")
        
        result = await conversation_manager.chat(
            user_message=message,
            use_context=True,
            show_reasoning=True
        )
        
        print(f"🤖 Assistant [{result['source']}] ({result['response_time']:.3f}s):")
        
        # Show routing analysis if available
        if result.get('analysis'):
            analysis = result['analysis']
            print(f"   📊 Complexity Score: {analysis.get('complexity_score', 'N/A')}/10")
            print(f"   🎯 Routing Target: {analysis.get('route_to', 'N/A')}")
        
        response_preview = result['assistant_response'][:300]
        print(f"   {response_preview}...")
        
        if not result['success']:
            print(f"   ⚠️ Response failed")
    
    # Scenario 3: Context-aware conversation
    print("\n" + "="*70)
    print("📝 SCENARIO 3: Context-Aware Conversation")
    print("="*70)
    
    # Clear context for fresh start
    conversation_manager.clear_context()
    
    scenario3_exchanges = [
        "I'm working on a hybrid AI routing system.",
        "What are the key benefits of this approach?",
        "How would you implement fallback mechanisms?",
        "Can you summarize what we've discussed about my system?"
    ]
    
    for i, message in enumerate(scenario3_exchanges, 1):
        print(f"\n💬 User ({i}/{len(scenario3_exchanges)}): {message}")
        
        result = await conversation_manager.chat(
            user_message=message,
            use_context=True,
            show_reasoning=False
        )
        
        print(f"🤖 Assistant [{result['source']}] ({result['response_time']:.3f}s):")
        print(f"   {result['assistant_response'][:250]}...")
        
        if not result['success']:
            print(f"   ⚠️ Response failed")
    
    # Scenario 4: Mixed complexity conversation
    print("\n" + "="*70)
    print("📝 SCENARIO 4: Mixed Complexity Chat")
    print("="*70)
    
    # Clear context
    conversation_manager.clear_context()
    
    scenario4_exchanges = [
        "What's 100 + 50?",
        "Analyze the economic implications of AI automation in manufacturing",
        "Thanks!",
        "Design a comprehensive strategy for implementing this in a Fortune 500 company"
    ]
    
    for i, message in enumerate(scenario4_exchanges, 1):
        print(f"\n💬 User ({i}/{len(scenario4_exchanges)}): {message}")
        
        result = await conversation_manager.chat(
            user_message=message,
            use_context=True,
            show_reasoning=True
        )
        
        print(f"🤖 Assistant [{result['source']}] ({result['response_time']:.3f}s):")
        
        if result.get('analysis'):
            analysis = result['analysis']
            print(f"   📊 Score: {analysis.get('complexity_score', 'N/A')}/10")
        
        print(f"   {result['assistant_response'][:200]}...")
        
        if not result['success']:
            print(f"   ⚠️ Response failed")
    
    print("\n" + "="*70)
    print("✅ All conversation scenarios completed!")
    print("="*70)
    
    # Display final analytics
    conversation_manager.display_conversation_analytics()

# Run the test scenarios
await test_conversation_scenarios()

## Step 5.5: Interactive Conversation Testing

Test the conversation manager interactively with custom queries.

In [ ]:
async def interactive_chat_demo():
    """Interactive chat demonstration."""
    
    if not conversation_manager:
        print("❌ Conversation manager not available")
        return
    
    print("\n🎮 Interactive Chat Demo with Agent Framework")
    print("="*70)
    
    # Start fresh conversation
    conversation_manager.clear_context()
    
    # Example conversation
    demo_queries = [
        "Hello! I'm interested in learning about hybrid AI systems.",
        "What makes them different from traditional cloud-only approaches?",
        "Can you give me a specific example of cost savings?",
        "How would I get started implementing this?"
    ]
    
    print("Running demo conversation with context awareness...\n")
    
    for i, query in enumerate(demo_queries, 1):
        print(f"{'='*70}")
        print(f"Exchange {i}/{len(demo_queries)}")
        print(f"{'='*70}")
        print(f"👤 User: {query}")
        
        result = await conversation_manager.chat(
            user_message=query,
            use_context=True,
            show_reasoning=True
        )
        
        print(f"\n🤖 Assistant [{result['source']}]:")
        print(f"   Response Time: {result['response_time']:.3f}s")
        
        if result.get('analysis'):
            analysis = result['analysis']
            print(f"   Complexity: {analysis.get('complexity_score', 'N/A')}/10")
            print(f"   Router: {analysis.get('router_used', 'N/A')}")
        
        print(f"\n   Response:")
        # Show full response for demo
        response_lines = result['assistant_response'].split('\n')
        for line in response_lines[:5]:  # Show first 5 lines
            print(f"   {line}")
        if len(response_lines) > 5:
            print(f"   ... ({len(response_lines) - 5} more lines)")
        
        print()
    
    # Show conversation history
    print(f"\n{'='*70}")
    print("📚 Conversation History Summary")
    print(f"{'='*70}")
    
    history = conversation_manager.get_conversation_history(count=10)
    print(f"Total exchanges tracked: {len(history)}")
    
    print("\n📊 Final Analytics:")
    conversation_manager.display_conversation_analytics()
    
    # Export conversation
    try:
        export_path = conversation_manager.export_conversation()
        print(f"\n💾 Conversation exported to: {export_path}")
    except Exception as e:
        print(f"\n⚠️ Export failed: {e}")

# Run interactive demo
await interactive_chat_demo()

## Step 5.6: Performance Analysis and Routing Insights

Analyze routing performance and provide insights.

In [ ]:
def analyze_routing_performance():
    """Analyze routing performance and provide insights."""
    
    if not conversation_manager:
        print("❌ Conversation manager not available")
        return
    
    print("\n📊 Routing Performance Analysis")
    print("="*70)
    
    summary = conversation_manager.get_session_summary()
    
    if 'message' in summary:
        print(summary['message'])
        return
    
    # Overall metrics
    print("\n🎯 Overall Performance:")
    print(f"   Total Conversations: {summary['total_exchanges']}")
    print(f"   Average Response Time: {summary['average_response_time']:.3f}s")
    print(f"   Total Session Time: {summary['total_time']:.2f}s")
    print(f"   Session Duration: {summary['session_duration']:.1f}s")
    
    # Routing efficiency
    print(f"\n🔀 Routing Efficiency:")
    print(f"   Model Switches: {summary['model_switches']}")
    
    if summary['total_exchanges'] > 0:
        switch_rate = (summary['model_switches'] / summary['total_exchanges']) * 100
        print(f"   Switch Rate: {switch_rate:.1f}%")
    
    # Distribution analysis
    print(f"\n📈 Routing Distribution Analysis:")
    
    dist = summary['routing_distribution']
    
    for target, data in dist.items():
        if data['count'] > 0:
            print(f"\n   {target.upper()}:")
            print(f"      Count: {data['count']}")
            print(f"      Percentage: {data['percentage']:.1f}%")
            print(f"      Avg Time: {summary['total_time'] / summary['total_exchanges']:.3f}s")
    
    # Recommendations
    print(f"\n💡 Optimization Recommendations:")
    
    local_pct = dist['local']['percentage']
    agent_pct = dist['agent']['percentage']
    
    if local_pct > 70:
        print("   ✅ Excellent local routing - maximizing speed and cost savings")
    elif local_pct > 40:
        print("   ✅ Good balance between local and cloud routing")
    else:
        print("   💡 Consider tuning complexity threshold for more local routing")
    
    if agent_pct > 0:
        print(f"   ✅ Agent Framework handling {agent_pct:.1f}% of complex queries")
    
    if summary['model_switches'] > summary['total_exchanges'] * 0.5:
        print("   💡 High switch rate - consider context-aware routing optimization")
    else:
        print("   ✅ Efficient model switching based on query complexity")
    
    print("\n" + "="*70)

# Run performance analysis
analyze_routing_performance()

## Step 5.7: Context Management Features

Demonstrate advanced context management capabilities.

In [ ]:
async def test_context_management():
    """Test context management features."""
    
    if not conversation_manager:
        print("❌ Conversation manager not available")
        return
    
    print("\n🧪 Testing Context Management Features")
    print("="*70)
    
    # Clear for fresh start
    conversation_manager.clear_context()
    
    print("\n📝 Building context across multiple exchanges...")
    
    # Build context
    context_building = [
        "My name is Alice and I work at TechCorp.",
        "We're building a financial analysis platform.",
        "It needs to process sensitive customer data.",
        "What security considerations should we keep in mind for my platform?"
    ]
    
    for i, msg in enumerate(context_building, 1):
        print(f"\n💬 Exchange {i}: {msg}")
        result = await conversation_manager.chat(msg, use_context=True)
        print(f"   ✅ Routed to: {result['source']}")
        print(f"   📝 Response preview: {result['assistant_response'][:100]}...")
    
    # Test context retrieval
    print(f"\n{'='*70}")
    print("📚 Context Retrieval Test")
    print(f"{'='*70}")
    
    history = conversation_manager.get_conversation_history(count=5)
    print(f"\nRetrieved {len(history)} recent exchanges:")
    
    for i, exchange in enumerate(history[-3:], 1):  # Show last 3
        print(f"\n   Exchange {i}:")
        print(f"      User: {exchange.get('user', 'N/A')[:60]}...")
        print(f"      Assistant: {exchange.get('assistant', 'N/A')[:60]}...")
    
    # Test context-aware response
    print(f"\n{'='*70}")
    print("🎯 Context-Aware Response Test")
    print(f"{'='*70}")
    
    context_query = "Based on our conversation, what's the most important thing for us to implement first?"
    print(f"\n💬 User: {context_query}")
    
    result = await conversation_manager.chat(context_query, use_context=True, show_reasoning=True)
    
    print(f"\n🤖 Assistant [{result['source']}]:")
    print(f"   This response should reference: Alice, TechCorp, financial platform, security")
    print(f"\n   {result['assistant_response'][:400]}...")
    
    # Test without context
    print(f"\n{'='*70}")
    print("🚫 No Context Test (for comparison)")
    print(f"{'='*70}")
    
    conversation_manager.clear_context()
    
    no_context_query = "What should we implement first?"
    print(f"\n💬 User: {no_context_query}")
    
    result = await conversation_manager.chat(no_context_query, use_context=False)
    
    print(f"\n🤖 Assistant [{result['source']}]:")
    print(f"   This response should be generic (no context available)")
    print(f"\n   {result['assistant_response'][:300]}...")
    
    print(f"\n{'='*70}")
    print("✅ Context management testing complete!")
    print(f"{'='*70}")

# Run context management tests
await test_context_management()

## 🎉 Lab 5 Complete!

### What You've Accomplished:

✅ **Agent Framework Integration**: Successfully integrated modern Agent Framework hybrid routing

✅ **Two-Tier Architecture**: Implemented Local → Agent Framework/APIM intelligent routing

✅ **Context Management**: Leveraged built-in ConversationContextManager for seamless context preservation

✅ **Multi-Turn Conversations**: Tested various conversation scenarios with context awareness

✅ **Performance Analytics**: Comprehensive routing statistics and performance insights

✅ **Production Ready**: Async/await patterns, error handling, and conversation persistence

### Key Features Demonstrated:

🤖 **Modern Agent Framework**: 
- AzureAIAgentClient for cloud routing
- Async/await patterns throughout
- No thread/run polling overhead

🏠 **Local Model Integration**:
- Fast responses for simple queries
- Cost-effective processing
- Privacy-preserving local inference

🧠 **ML-Powered Routing**:
- BERT/PHI query analysis
- Intelligent complexity scoring
- Context-aware decisions

📊 **Enterprise Analytics**:
- Routing distribution tracking
- Performance metrics
- Cost optimization insights

### Routing Performance:

**Local Model** (Simple queries):
- Response Time: <100ms
- Cost: Zero
- Use Cases: Greetings, simple Q&A, calculations

**Agent Framework** (Complex queries):
- Response Time: 1-3s
- Cost: API usage
- Use Cases: Analysis, design, strategic planning

**APIM Router** (Enterprise queries):
- Response Time: 500ms-2s
- Cost: Managed API costs
- Use Cases: Enterprise workloads, compliance requirements

### Context Management:

✅ **Automatic Context Tracking**: All exchanges preserved in ConversationContextManager

✅ **Context-Aware Routing**: Query analysis considers conversation history

✅ **Session Management**: Unique session IDs for conversation isolation

✅ **Export/Import**: Save and resume conversations

### Best Practices Demonstrated:

1. **Async Patterns**: All I/O operations use async/await for efficiency
2. **Error Handling**: Graceful fallbacks at each routing tier
3. **Context Preservation**: Seamless context across model switches
4. **Analytics**: Built-in telemetry for monitoring and optimization
5. **Production Ready**: Proper logging, error messages, and status reporting

### Next Steps:

🚀 **Deploy to Production**:
- Add monitoring and alerting
- Implement caching for frequent queries
- Set up conversation persistence storage

🔧 **Customize**:
- Adjust complexity thresholds for your use case
- Add custom routing rules
- Integrate domain-specific tools

📈 **Optimize**:
- Fine-tune ML routers with your data
- Implement query caching
- Add response streaming

**Your hybrid AI orchestration system with Agent Framework is production-ready!** 🎉

*This lab demonstrates the modern approach to building intelligent, context-aware hybrid AI systems using Microsoft's Agent Framework with Azure AI Foundry.*